# Post-Processing Normalization (SM+P2+P3=1)

**Problem**: The CV-LB gap is ~10x. The intercept (0.0533) is 3x higher than target (0.01727).

**Key Insight**: Other competitors use post-processing normalization (SM+P2+P3=1) but we haven't tried it!

**Hypothesis**: Enforcing mass balance constraint may improve generalization.

**Implementation**:
- Use best model from exp_026 (weighted loss [1,1,2])
- Add post-processing normalization: preds = preds / preds.sum(axis=1)
- This enforces SM + P2 + P3 = 1 (mass balance)

**Baseline**: exp_026 CV 0.008465, LB 0.0887

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')
print(f'Total features: 5 (kinetic) + {SPANGE_DF.shape[1]} (Spange) + {DRFP_FILTERED.shape[1]} (DRFP) + {ACS_PCA_DF.shape[1]} (ACS PCA) = {5 + SPANGE_DF.shape[1] + DRFP_FILTERED.shape[1] + ACS_PCA_DF.shape[1]}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)
Total features: 5 (kinetic) + 13 (Spange) + 122 (DRFP) + 5 (ACS PCA) = 145


In [4]:
# Combined Featurizer
class ACSPCAFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'Feature dimension: {ACSPCAFeaturizer().feats_dim}')

Feature dimension: 145


In [5]:
# Weighted Huber Loss
class WeightedHuberLoss(nn.Module):
    def __init__(self, weights=[1.0, 1.0, 2.0]):
        super().__init__()
        self.weights = torch.tensor(weights, dtype=torch.double)
        self.huber = nn.HuberLoss(reduction='none')
    
    def forward(self, pred, target):
        huber_loss = self.huber(pred, target)
        weighted_loss = huber_loss * self.weights.to(pred.device)
        return weighted_loss.mean()

print('WeightedHuberLoss defined')

WeightedHuberLoss defined


In [6]:
# MLP Model
class MLPModelInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16], output_dim=3, dropout=0.05):
        super(MLPModelInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h_dim), nn.BatchNorm1d(h_dim), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h_dim
        layers.extend([nn.Linear(prev_dim, output_dim), nn.Sigmoid()])
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

print('MLPModelInternal defined')

MLPModelInternal defined


In [7]:
# MLP Ensemble with Weighted Loss
class WeightedMLPEnsemble:
    def __init__(self, hidden_dims=[32, 16], n_models=5, data='single', loss_weights=[1.0, 1.0, 2.0]):
        self.hidden_dims = hidden_dims
        self.n_models = n_models
        self.data_type = data
        self.loss_weights = loss_weights
        self.featurizer = ACSPCAFeaturizer(mixed=(data=='full'))
        self.models = []

    def train_model(self, X_train, y_train, epochs=200, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize_torch(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all, y_all = X_std, y_vals
            
        input_dim = X_all.shape[1]
        self.models = []
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i * 13)
            np.random.seed(42 + i * 13)
            
            model = MLPModelInternal(input_dim, self.hidden_dims).to(device).double()
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
            criterion = WeightedHuberLoss(weights=self.loss_weights)
            
            dataset = TensorDataset(X_all.to(device), y_all.to(device))
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            model.train()
            for epoch in range(epochs):
                epoch_loss = 0.0
                for batch_X, batch_y in loader:
                    optimizer.zero_grad()
                    pred = model(batch_X)
                    loss = criterion(pred, batch_y)
                    loss.backward()
                    optimizer.step()
                    epoch_loss += loss.item()
                scheduler.step(epoch_loss / len(loader))
            
            model.eval()
            self.models.append(model)

    def predict(self, X_test):
        X_feat = self.featurizer.featurize_torch(X_test, flip=False).to(device)
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_test, flip=True).to(device)
        
        all_preds = []
        with torch.no_grad():
            for model in self.models:
                pred = model(X_feat)
                if self.data_type == 'full':
                    pred_flip = model(X_flip)
                    pred = (pred + pred_flip) / 2
                all_preds.append(pred)
        
        return torch.stack(all_preds).mean(dim=0).cpu()

print('WeightedMLPEnsemble defined')

WeightedMLPEnsemble defined


In [8]:
# LightGBM Wrapper
class LGBMWrapper:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = ACSPCAFeaturizer(mixed=(data=='full'))
        self.models = []

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.models = []
        params = {'objective': 'regression', 'metric': 'mse', 'boosting_type': 'gbdt',
                  'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.9,
                  'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'seed': 42}
        
        for i in range(3):
            train_data = lgb.Dataset(X_all, label=y_all[:, i])
            model = lgb.train(params, train_data, num_boost_round=100)
            self.models.append(model)

    def predict(self, X_test):
        X_feat = self.featurizer.featurize(X_test, flip=False)
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_test, flip=True)
        
        preds = []
        for i, model in enumerate(self.models):
            pred = model.predict(X_feat)
            if self.data_type == 'full':
                pred_flip = model.predict(X_flip)
                pred = (pred + pred_flip) / 2
            preds.append(pred)
        
        return torch.tensor(np.column_stack(preds))

print('LGBMWrapper defined')

LGBMWrapper defined


In [9]:
# Normalization Ensemble: MLP + LightGBM with POST-PROCESSING NORMALIZATION
class NormalizationEnsemble:
    def __init__(self, data='single'):
        self.data_type = data
        self.mlp = WeightedMLPEnsemble(hidden_dims=[32, 16], n_models=5, data=data, loss_weights=[1.0, 1.0, 2.0])
        self.lgbm = LGBMWrapper(data=data)
        self.mlp_weight = 0.6
        self.lgbm_weight = 0.4

    def train_model(self, X_train, y_train):
        self.mlp.train_model(X_train, y_train)
        self.lgbm.train_model(X_train, y_train)

    def predict(self, X_test):
        mlp_pred = self.mlp.predict(X_test)
        lgbm_pred = self.lgbm.predict(X_test)
        
        # Weighted average
        combined = self.mlp_weight * mlp_pred + self.lgbm_weight * lgbm_pred
        
        # Clip to [0, 1]
        combined = torch.clamp(combined, 0, 1)
        
        # POST-PROCESSING NORMALIZATION: SM + P2 + P3 = 1
        # preds is [P2, P3, SM] - normalize so they sum to 1
        row_sums = combined.sum(dim=1, keepdim=True)
        row_sums = torch.clamp(row_sums, min=1e-6)  # Avoid division by zero
        normalized = combined / row_sums
        
        return normalized

print('NormalizationEnsemble defined with POST-PROCESSING NORMALIZATION (SM+P2+P3=1)')

NormalizationEnsemble defined with POST-PROCESSING NORMALIZATION (SM+P2+P3=1)


In [10]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = NormalizationEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:49, 49.70s/it]

2it [01:38, 49.37s/it]

3it [02:25, 48.20s/it]

4it [03:12, 47.55s/it]

...

21it [17:05, 48.91s/it]

22it [17:54, 48.93s/it]

23it [18:42, 48.82s/it]

24it [19:31, 48.84s/it]

24it [19:31, 48.81s/it]

In [11]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = NormalizationEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [02:49, 169.88s/it]

2it [05:36, 168.22s/it]

3it [08:29, 170.36s/it]

4it [11:17, 169.35s/it]

...

10it [28:30, 174.94s/it]

11it [31:54, 183.76s/it]

12it [35:19, 190.16s/it]

13it [38:45, 195.00s/it]

13it [38:45, 178.86s/it]

In [12]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [13]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

# Check normalization constraint
pred_sums_single = preds_single.sum(axis=1)
pred_sums_full = preds_full.sum(axis=1)
print(f'\n=== NORMALIZATION CHECK ===')
print(f'Single Solvent: pred sums range [{pred_sums_single.min():.4f}, {pred_sums_single.max():.4f}], mean={pred_sums_single.mean():.4f}')
print(f'Full Data: pred sums range [{pred_sums_full.min():.4f}, {pred_sums_full.max():.4f}], mean={pred_sums_full.mean():.4f}')

print(f'\n=== CV SCORE VERIFICATION ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nexp_026 baseline (no normalization): CV 0.008465, LB 0.0887')
print(f'This (with normalization): CV {overall_mse:.6f}')

if overall_mse < 0.008465:
    improvement = (0.008465 - overall_mse) / 0.008465 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_026!')
else:
    degradation = (overall_mse - 0.008465) / 0.008465 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_026')


=== NORMALIZATION CHECK ===
Single Solvent: pred sums range [1.0000, 1.0000], mean=1.0000
Full Data: pred sums range [1.0000, 1.0000], mean=1.0000

=== CV SCORE VERIFICATION ===
Single Solvent MSE: 0.016141 (n=656)
Full Data MSE: 0.016201 (n=1227)
Overall MSE: 0.016180

exp_026 baseline (no normalization): CV 0.008465, LB 0.0887
This (with normalization): CV 0.016180

✗ WORSE: 91.14% worse than exp_026


In [14]:
# Check actual target sums in the data
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

actual_sums_single = Y_single.values.sum(axis=1)
actual_sums_full = Y_full.values.sum(axis=1)

print(f'=== ACTUAL TARGET SUMS ===')
print(f'Single Solvent: range [{actual_sums_single.min():.4f}, {actual_sums_single.max():.4f}], mean={actual_sums_single.mean():.4f}, std={actual_sums_single.std():.4f}')
print(f'Full Data: range [{actual_sums_full.min():.4f}, {actual_sums_full.max():.4f}], mean={actual_sums_full.mean():.4f}, std={actual_sums_full.std():.4f}')
print(f'\\nThe actual targets do NOT sum to exactly 1.0!')

=== ACTUAL TARGET SUMS ===
Single Solvent: range [0.0288, 1.0000], mean=0.7955, std=0.1942
Full Data: range [0.0112, 1.1233], mean=0.8035, std=0.2091
\nThe actual targets do NOT sum to exactly 1.0!
